In [1]:
%load_ext autotime

import os, math

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from tqdm import tqdm, trange

In [2]:
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, Input, BatchNormalization
from sklearn.metrics import mean_squared_error, mean_absolute_error

def model(test_data, train_data):
    test_time = test_data['time']
    test_gl_value = test_data['gl_value']
    
    test_data.drop(columns = ['time'], inplace = True)
    train_data.rename(columns = {'bolus_type_normal dual':'bolus_type_normal_dual', 'bolus_type_square dual':'bolus_type_square_dual'}, inplace = True)
    train_data.drop(columns = ['time'], inplace = True)
    
    empty_train_col = [0]*len(train_data)
    for i, item in enumerate(test_data.columns):
        if item not in train_data.columns:
            train_data.insert(i, item, empty_train_col)

    empty_test_col = [0]*len(test_data)
    for i, item in enumerate(train_data.columns):
        if item not in test_data.columns:
            test_data.insert(i, item, empty_test_col)
    
    X_data = train_data.drop(columns = ['gl_predict'])
    y_data = train_data[['gl_predict']]
    input_dim = X_data.shape[1]
    
    scaler_x = MinMaxScaler()
    scaler_y = MinMaxScaler()
    scaler_x.fit(X_data)
    scaler_y.fit(y_data)
    scaled_X_train_data = scaler_x.transform(X_data)
    scaled_y_train_data = scaler_y.transform(y_data)
    scaled_X_train_data = np.reshape(scaled_X_train_data, (scaled_X_train_data.shape[0], 1, scaled_X_train_data.shape[1]))
    
    model = Sequential()
    model.add(LSTM(128, input_shape = (scaled_X_train_data.shape[1], scaled_X_train_data.shape[2])))
    model.add(Dense(150, activation = 'relu'))
    model.add(Dropout(0.20))
    model.add(Dense(100, activation = 'relu'))
    model.add(Dropout(0.15))
    model.add(Dense(50, activation = 'relu'))
    model.add(Dense(20, activation = 'relu'))
    model.add(Dense(1, activation = 'relu'))
    model.compile(loss = 'mse', optimizer = 'adam')
    model.fit(scaled_X_train_data, scaled_y_train_data, epochs = 200, batch_size = 32, shuffle = False)
    
    X_test_data = test_data.drop(columns = ['gl_predict'])
    y_test_data = test_data[['gl_predict']]
    scaled_X_test_data = scaler_x.transform(X_test_data)
    scaled_X_test_data = np.reshape(scaled_X_test_data, (scaled_X_test_data.shape[0], 1, scaled_X_test_data.shape[1]))
    prediction = model.predict(scaled_X_test_data, batch_size = 32)
    scaled_prediction = scaler_y.inverse_transform(prediction)
    
    final_output = pd.DataFrame(columns = ['timestamp', 'predicted_BGL_value'])
    final_output['timestamp'] = test_time
#     final_output['true_BGL_value'] = test_gl_value
    final_output['predicted_BGL_value'] = scaled_prediction
    
    return (final_output, math.sqrt(mean_squared_error(scaled_prediction, y_test_data)), mean_absolute_error(scaled_prediction, y_test_data))

Using TensorFlow backend.


time: 1.27 s


In [3]:
test_data_list = ['540-ws-training.csv', '544-ws-training.csv', '552-ws-training.csv', '567-ws-training.csv', '584-ws-training.csv', '596-ws-training.csv']

rmse_list = {}
mae_list = {}
for file in os.listdir('../data/derived/60_min_complete_dataset/'):
    if file in test_data_list:
        train_data = pd.read_csv('../data/derived/60_min_complete_dataset/'+file)
        test_data = pd.read_csv('../data/derived_test/60_min_complete_dataset/'+file[0:7]+'testing.csv')
        final_output, rmse_error, mae_error = model(test_data, train_data)
        rmse_list[file] = rmse_error
        mae_list[file] = mae_error
        print ('{} - {}'.format(file, rmse_error))
        print ('{} - {}'.format(file, mae_error))
        final_output.to_csv('../data/output/lstm/'+file[0:7]+'result_60min.csv')

Epoch 1/200
1103/1103 [==============================] - 0s 406us/step - loss: 0.0561
Epoch 2/200
1103/1103 [==============================] - 0s 73us/step - loss: 0.0229
Epoch 3/200
1103/1103 [==============================] - 0s 74us/step - loss: 0.0188
Epoch 4/200
1103/1103 [==============================] - 0s 69us/step - loss: 0.0164
Epoch 5/200
1103/1103 [==============================] - 0s 67us/step - loss: 0.0141
Epoch 6/200
1103/1103 [==============================] - 0s 69us/step - loss: 0.0144
Epoch 7/200
1103/1103 [==============================] - 0s 78us/step - loss: 0.0139
Epoch 8/200
1103/1103 [==============================] - 0s 80us/step - loss: 0.0134
Epoch 9/200
1103/1103 [==============================] - 0s 80us/step - loss: 0.0152
Epoch 10/200
1103/1103 [==============================] - 0s 77us/step - loss: 0.0136
Epoch 11/200
1103/1103 [==============================] - 0s 81us/step - loss: 0.0128
Epoch 12/200
1103/1103 [==============================] - 0s 8

1103/1103 [==============================] - 0s 81us/step - loss: 0.0099
Epoch 97/200
1103/1103 [==============================] - 0s 77us/step - loss: 0.0098
Epoch 98/200
1103/1103 [==============================] - 0s 74us/step - loss: 0.0104
Epoch 99/200
1103/1103 [==============================] - 0s 75us/step - loss: 0.0098
Epoch 100/200
1103/1103 [==============================] - 0s 74us/step - loss: 0.0102
Epoch 101/200
1103/1103 [==============================] - 0s 73us/step - loss: 0.0094
Epoch 102/200
1103/1103 [==============================] - 0s 74us/step - loss: 0.0097
Epoch 103/200
1103/1103 [==============================] - 0s 74us/step - loss: 0.0098
Epoch 104/200
1103/1103 [==============================] - 0s 75us/step - loss: 0.0097
Epoch 105/200
1103/1103 [==============================] - 0s 74us/step - loss: 0.0098
Epoch 106/200
1103/1103 [==============================] - 0s 75us/step - loss: 0.0102
Epoch 107/200
1103/1103 [==============================] - 0

1103/1103 [==============================] - 0s 85us/step - loss: 0.0086
Epoch 191/200
1103/1103 [==============================] - 0s 97us/step - loss: 0.0080
Epoch 192/200
1103/1103 [==============================] - 0s 92us/step - loss: 0.0082
Epoch 193/200
1103/1103 [==============================] - 0s 101us/step - loss: 0.0088
Epoch 194/200
1103/1103 [==============================] - 0s 100us/step - loss: 0.0084
Epoch 195/200
1103/1103 [==============================] - 0s 104us/step - loss: 0.0096
Epoch 196/200
1103/1103 [==============================] - 0s 102us/step - loss: 0.0094
Epoch 197/200
1103/1103 [==============================] - 0s 98us/step - loss: 0.0097
Epoch 198/200
1103/1103 [==============================] - 0s 109us/step - loss: 0.0088
Epoch 199/200
1103/1103 [==============================] - 0s 111us/step - loss: 0.0082
Epoch 200/200
1103/1103 [==============================] - 0s 109us/step - loss: 0.0088
584-ws-training.csv - 42.54581161224805
584-ws-tra

1127/1127 [==============================] - 0s 106us/step - loss: 0.0088
Epoch 84/200
1127/1127 [==============================] - 0s 113us/step - loss: 0.0090
Epoch 85/200
1127/1127 [==============================] - 0s 104us/step - loss: 0.0091
Epoch 86/200
1127/1127 [==============================] - 0s 109us/step - loss: 0.0090
Epoch 87/200
1127/1127 [==============================] - 0s 104us/step - loss: 0.0088
Epoch 88/200
1127/1127 [==============================] - 0s 102us/step - loss: 0.0090
Epoch 89/200
1127/1127 [==============================] - 0s 105us/step - loss: 0.0091
Epoch 90/200
1127/1127 [==============================] - 0s 105us/step - loss: 0.0089
Epoch 91/200
1127/1127 [==============================] - 0s 115us/step - loss: 0.0086
Epoch 92/200
1127/1127 [==============================] - 0s 124us/step - loss: 0.0090
Epoch 93/200
1127/1127 [==============================] - 0s 103us/step - loss: 0.0088
Epoch 94/200
1127/1127 [==============================] 

1127/1127 [==============================] - 0s 129us/step - loss: 0.0079
Epoch 177/200
1127/1127 [==============================] - 0s 126us/step - loss: 0.0079
Epoch 178/200
1127/1127 [==============================] - 0s 124us/step - loss: 0.0077
Epoch 179/200
1127/1127 [==============================] - 0s 121us/step - loss: 0.0074
Epoch 180/200
1127/1127 [==============================] - 0s 130us/step - loss: 0.0079
Epoch 181/200
1127/1127 [==============================] - 0s 127us/step - loss: 0.0076
Epoch 182/200
1127/1127 [==============================] - 0s 122us/step - loss: 0.0077
Epoch 183/200
1127/1127 [==============================] - 0s 123us/step - loss: 0.0077
Epoch 184/200
1127/1127 [==============================] - 0s 126us/step - loss: 0.0075
Epoch 185/200
1127/1127 [==============================] - 0s 143us/step - loss: 0.0076
Epoch 186/200
1127/1127 [==============================] - 0s 142us/step - loss: 0.0076
Epoch 187/200
1127/1127 [=====================

...

924/924 [==============================] - 0s 193us/step - loss: 0.0101
Epoch 152/200
924/924 [==============================] - 0s 170us/step - loss: 0.0098
Epoch 153/200
924/924 [==============================] - 0s 200us/step - loss: 0.0110
Epoch 154/200
924/924 [==============================] - 0s 202us/step - loss: 0.0112
Epoch 155/200
924/924 [==============================] - 0s 206us/step - loss: 0.0101
Epoch 156/200
924/924 [==============================] - 0s 162us/step - loss: 0.0087
Epoch 157/200
924/924 [==============================] - 0s 210us/step - loss: 0.0092
Epoch 158/200
924/924 [==============================] - 0s 202us/step - loss: 0.0095 0s - loss: 0.00
Epoch 159/200
924/924 [==============================] - 0s 184us/step - loss: 0.0099
Epoch 160/200
924/924 [==============================] - 0s 186us/step - loss: 0.0098
Epoch 161/200
924/924 [==============================] - 0s 189us/step - loss: 0.0093
Epoch 162/200
924/924 [=============================

1055/1055 [==============================] - 0s 155us/step - loss: 0.0078
Epoch 45/200
1055/1055 [==============================] - 0s 157us/step - loss: 0.0075
Epoch 46/200
1055/1055 [==============================] - 0s 187us/step - loss: 0.0077
Epoch 47/200
1055/1055 [==============================] - 0s 167us/step - loss: 0.0076
Epoch 48/200
1055/1055 [==============================] - 0s 167us/step - loss: 0.0081
Epoch 49/200
1055/1055 [==============================] - 0s 183us/step - loss: 0.0077
Epoch 50/200
1055/1055 [==============================] - 0s 178us/step - loss: 0.0081
Epoch 51/200
1055/1055 [==============================] - 0s 206us/step - loss: 0.0076
Epoch 52/200
1055/1055 [==============================] - 0s 202us/step - loss: 0.0081
Epoch 53/200
1055/1055 [==============================] - 0s 207us/step - loss: 0.0077
Epoch 54/200
1055/1055 [==============================] - 0s 218us/step - loss: 0.0082
Epoch 55/200
1055/1055 [==============================] 

1055/1055 [==============================] - 0s 210us/step - loss: 0.0066
Epoch 138/200
1055/1055 [==============================] - 0s 204us/step - loss: 0.0066
Epoch 139/200
1055/1055 [==============================] - 0s 208us/step - loss: 0.0066
Epoch 140/200
1055/1055 [==============================] - 0s 217us/step - loss: 0.0066
Epoch 141/200
1055/1055 [==============================] - 0s 181us/step - loss: 0.0066
Epoch 142/200
1055/1055 [==============================] - 0s 187us/step - loss: 0.0066
Epoch 143/200
1055/1055 [==============================] - 0s 132us/step - loss: 0.0064
Epoch 144/200
1055/1055 [==============================] - 0s 131us/step - loss: 0.0067
Epoch 145/200
1055/1055 [==============================] - 0s 136us/step - loss: 0.0063
Epoch 146/200
1055/1055 [==============================] - 0s 125us/step - loss: 0.0067
Epoch 147/200
1055/1055 [==============================] - 0s 120us/step - loss: 0.0064
Epoch 148/200
1055/1055 [=====================

1092/1092 [==============================] - 0s 237us/step - loss: 0.0129
Epoch 30/200
1092/1092 [==============================] - 0s 239us/step - loss: 0.0131
Epoch 31/200
1092/1092 [==============================] - 0s 233us/step - loss: 0.0135
Epoch 32/200
1092/1092 [==============================] - 0s 238us/step - loss: 0.0125
Epoch 33/200
1092/1092 [==============================] - 0s 210us/step - loss: 0.0123
Epoch 34/200
1092/1092 [==============================] - 0s 199us/step - loss: 0.0127
Epoch 35/200
1092/1092 [==============================] - 0s 227us/step - loss: 0.0129
Epoch 36/200
1092/1092 [==============================] - 0s 216us/step - loss: 0.0127
Epoch 37/200
1092/1092 [==============================] - 0s 231us/step - loss: 0.0123
Epoch 38/200
1092/1092 [==============================] - 0s 184us/step - loss: 0.0121
Epoch 39/200
1092/1092 [==============================] - 0s 215us/step - loss: 0.0121
Epoch 40/200
1092/1092 [==============================] 

1092/1092 [==============================] - 0s 230us/step - loss: 0.0097
Epoch 123/200
1092/1092 [==============================] - 0s 300us/step - loss: 0.0100
Epoch 124/200
1092/1092 [==============================] - 0s 164us/step - loss: 0.0095
Epoch 125/200
1092/1092 [==============================] - 0s 168us/step - loss: 0.0095
Epoch 126/200
1092/1092 [==============================] - 0s 196us/step - loss: 0.0103
Epoch 127/200
1092/1092 [==============================] - 0s 204us/step - loss: 0.0091
Epoch 128/200
1092/1092 [==============================] - 0s 212us/step - loss: 0.0095
Epoch 129/200
1092/1092 [==============================] - 0s 217us/step - loss: 0.0091
Epoch 130/200
1092/1092 [==============================] - 0s 188us/step - loss: 0.0095
Epoch 131/200
1092/1092 [==============================] - 0s 239us/step - loss: 0.0095
Epoch 132/200
1092/1092 [==============================] - 0s 221us/step - loss: 0.0096
Epoch 133/200
1092/1092 [=====================

In [4]:
rmse_list

{'584-ws-training.csv': 42.54581161224805,
 '567-ws-training.csv': 37.847341482889746,
 '596-ws-training.csv': 30.068422035835376,
 '552-ws-training.csv': 43.08559089705407,
 '544-ws-training.csv': 29.1580968067276,
 '540-ws-training.csv': 49.57090330075394}

time: 8.07 ms


In [5]:
mae_list

{'584-ws-training.csv': 32.18255416627914,
 '567-ws-training.csv': 26.963814951272916,
 '596-ws-training.csv': 22.110684900192986,
 '552-ws-training.csv': 34.010134765871015,
 '544-ws-training.csv': 20.80687083624443,
 '540-ws-training.csv': 37.41165391097869}

time: 3.59 ms


In [6]:
from statistics import pstdev, mean

rmse_values = rmse_list.values()
mae_values = mae_list.values()

print ('Mean - RMSE : {}'.format(mean(rmse_values)))
print ('Standard Deviation - RMSE : {}'.format(pstdev(rmse_values)))

print ('Mean - MAE : {}'.format(mean(mae_values)))
print ('Standard Deviation - MAE : {}'.format(pstdev(mae_values)))

Mean - RMSE : 38.71269435591813
Standard Deviation - RMSE : 7.285583108847801
Mean - MAE : 28.914285588473195
Standard Deviation - MAE : 6.119341178326629
time: 21.2 ms
